In [ ]:
# Импортируем Pandas
import pandas as pd

# прочитаем наш csv-файл в DataFrame
melb_data = pd.read_csv('data/melb_data_fe.csv', sep=',')
# выведем первые пять строк таблицы
melb_data.head()

In [ ]:
# Информация о столбцах
display(melb_data.info())

In [8]:
# Преобразуем столбец Date в формат datetime
melb_data['Date'] = pd.to_datetime(melb_data['Date'], dayfirst=True)
# Находим второй по популярности квартал продажи
# и число объектов, проданных в этом квартале.
melb_data['Date'].dt.quarter.value_counts()


3    4873
2    4359
4    2329
1    2019
Name: Date, dtype: int64

In [ ]:
# Сделаем преобразование столбцов к типу данных category:
# список столбцов, которые мы не берём во внимание
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] 
# задаём максимальное число уникальных категорий
max_unique_count = 150 
# цикл по именам столбцов
for col in melb_data.columns: 
    # проверяем условие
    if melb_data[col].nunique() < max_unique_count and col not in cols_to_exclude:  
        # преобразуем тип столбца
        melb_data[col] = melb_data[col].astype('category')
display(melb_data.info())


## Сортировка данных в DataFrame

**МЕТОД SORT_VALUES()**

Сортировка по значениям одного столбца

In [ ]:
# Отсортируем таблицу по возрастанию цены объектов недвижимости (Price):
melb_data.sort_values(by='Price').head(10)

In [ ]:
# Oтсортируем таблицу по убыванию (от самой последней до самой первой) даты продажи объекта (Date). 
# Для этого выставим параметр ascending на False:
melb_data.sort_values(by='Date', ascending=False)

Сортировка по значениям нескольких столбцов 

Для сортировки по значениям нескольких столбцов необходимо передать названия этих столбцов в параметр **by**

Отсортируем таблицу сначала по возрастанию расстояния от центра города (Distance), а затем — по возрастанию цены объекта (Price). Для того чтобы вывод был более наглядным, выделим каждую десятую строку из столбцов Distance и Price результирующей таблицы:

In [ ]:
melb_data.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

In [14]:
# Результат изменится если поменятть порядок для by местами
melb_data.sort_values(by=['Price', 'Distance']).loc[::10, ['Distance', 'Price']]

,Distance,Price
2652,6.4,85000.0
7116,7.8,200000.0
5206,11.2,220000.0
7240,8.2,241000.0
2898,13.0,250000.0
...,...,...
13341,6.3,4060000.0
8591,1.5,4275000.0
1094,11.2,4700000.0
5801,6.1,5046000.0


**Комбинирование сортировки с фильтрацией**

In [15]:
# Найдём информацию, где коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8.
mask1 = melb_data['AreaRatio'] < -0.8
# Найдём информацию о таунхаусах (Type)
mask2 = melb_data['Type'] == 'townhouse'
# Найдём информацию по компаниии (SellerG) McGrath
mask3 = melb_data['SellerG'] == 'McGrath'
# Результат отсортируем по дате продажи (Date) в порядке возрастания, 
# а после проведём сортировку по убыванию коэффициента соотношения площадей. 
# Также обновим старые индексы на новые, установив параметр ignore_index на True. 
# Для наглядности результата выберем из таблицы только столбцы Data и AreaRatio:
melb_data[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

,Date,AreaRatio
0,2016-07-26,-0.974922
1,2016-09-24,-0.971831
2,2016-11-27,-0.953608
3,2016-12-11,-0.945946
4,2017-08-04,-0.947368
5,2017-08-04,-0.970874


**Важно** не сочетать фильтрацию и метод sort_values() с параметром inplace=True
**Лучше** использовать метод copy()

In [ ]:
# Создадим переменную с фильтром, где Rooms > 5
filtered = melb_data[melb_data['Rooms'] > 5].copy()
# Теперь используем sort_values()
filtered.sort_values(inplace=True, by=['Rooms'])
# Выведем первые пять строк
filtered.head()

In [18]:
melb_data.sort_values(
    # Производим сортировку столбца AreaRatio по убыванию
    by='AreaRatio',
    ascending=False,
    # индексы полученной таблицы заменим на новые
    ignore_index=True
# Извлекаем значение столбца BuildingArea в строке с индексом 1558
).loc[1558, 'BuildingArea']

126.0

In [24]:
# Создадим переменную с фильтром 'townhouse'
mask1 = melb_data['Type'] == 'townhouse'
# Создадим переменную с фильтром, где Rooms > 2
mask2 = melb_data['Rooms'] > 2
# Теперь сортируем полученную таблицу сначала по возрастанию числа комнат, 
# а затем по убыванию средней площади комнат (MeanRoomsSquare)
int(melb_data[mask1 & mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, 'Price'])

1300000

## Группировка данных в DataFrame. МЕТОД GROUPBY()

Метод groupby() возвращает объект DataFrameGroupBy, который хранит в себе информацию о том, какие строки относятся к определённой группе, и сам по себе не представляет для нас интереса. 

Однако к этому объекту можно применять уже знакомые нам агрегирующие методы (mean, median, sum и т. д.), чтобы рассчитывать показатели внутри каждой группы.

**ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ**

In [ ]:
# Группируем столбец типа объекта недвижимости (Type):
melb_data.groupby(by='Type').mean()

In [ ]:
# Преобразуем тип объекта в качестве отдельного столбца таблицы:
melb_data.groupby(by='Type', as_index=False).mean()

In [ ]:
# Выводим данные интересующего нас столбца Type и сравним цены:
melb_data.groupby('Type')['Price'].mean()

In [ ]:
# Hайдём минимальное значение расстояния от центра города до объекта в зависимости от его региона. 
# Результат отсортируем по убыванию расстояния:
melb_data.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

**ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ**

Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом agg(), который принимает список строк с названиями агрегаций.

In [ ]:
# Построим таблицу для анализа продаж по месяцам. 
# Значения цен объектов недвижимости (Price), 
# сгруппированных по номеру месяца продажи (MonthSale). 
melb_data.groupby('MonthSale')['Price'].agg(
    # Найдём количество продаж(count), 
    ['count', 
     # Найдём среднее значениe
     'mean', 
     # Найдём максимальное значениe
     'max']
# Результат отсортируем по количеству продаж в порядке убывания:
).sort_values(by='count', ascending=False)

In [33]:
# Найдём полную информация обо всех основных статистических характеристиках внутри каждой группы
melb_data.groupby('MonthSale')['Price'].agg('describe')

,count,mean,std,min,25%,50%,75%,max
MonthSale,,,,,,,,
1,278.0,9.397921e+05,577668.924214,170000.0,570500.0,795000.0,1111250.0,5200000.0
2,333.0,1.169051e+06,671564.357417,131000.0,710000.0,1020000.0,1478000.0,4735000.0
3,1408.0,1.146762e+06,709573.596867,85000.0,680000.0,945000.0,1400000.0,5600000.0
4,1246.0,1.050479e+06,591892.902979,145000.0,655000.0,905500.0,1298750.0,5500000.0
5,1644.0,1.097807e+06,668492.867996,145000.0,650000.0,905000.0,1371250.0,8000000.0
6,1469.0,1.068981e+06,606010.069052,222000.0,660000.0,900000.0,1325000.0,7650000.0
7,1835.0,9.314698e+05,537390.803161,190000.0,586750.0,800000.0,1150000.0,9000000.0
8,1850.0,1.056371e+06,619617.476541,160000.0,635000.0,892000.0,1310000.0,6500000.0
9,1188.0,1.126349e+06,608734.690742,170000.0,725000.0,980000.0,1360000.0,6400000.0


**Подсчёт числа уникальных значений.** 

Можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода agg() строку 'nunique'. 

Передадим дополнительно встроенную функцию set, чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов:

In [38]:
melb_data.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

,nunique,set
Regionname,,
Eastern Metropolitan,26,"{HAR, Woodards, Miles, RW, Fletchers, C21, hoc..."
Eastern Victoria,11,"{other, Ray, hockingstuart, HAR, Harcourts, Mc..."
Northern Metropolitan,40,"{Rendina, Hodges, HAR, Cayzer, Woodards, Burnh..."
Northern Victoria,11,"{other, Ray, Raine, hockingstuart, HAR, McDona..."
South-Eastern Metropolitan,25,"{Hodges, HAR, Woodards, RW, Fletchers, C21, Th..."
Southern Metropolitan,38,"{Rendina, Hodges, HAR, Cayzer, Woodards, RW, F..."
Western Metropolitan,34,"{Rendina, Hodges, Bells, HAR, Woodards, Burnha..."
Western Victoria,6,"{other, Ray, hockingstuart, Raine, HAR, YPA}"


In [39]:
melb_data.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
).sort_values(by='nunique', ascending=False)

,nunique,set
Regionname,,
Northern Metropolitan,40,"{Rendina, Hodges, HAR, Cayzer, Woodards, Burnh..."
Southern Metropolitan,38,"{Rendina, Hodges, HAR, Cayzer, Woodards, RW, F..."
Western Metropolitan,34,"{Rendina, Hodges, Bells, HAR, Woodards, Burnha..."
Eastern Metropolitan,26,"{HAR, Woodards, Miles, RW, Fletchers, C21, hoc..."
South-Eastern Metropolitan,25,"{Hodges, HAR, Woodards, RW, Fletchers, C21, Th..."
Eastern Victoria,11,"{other, Ray, hockingstuart, HAR, Harcourts, Mc..."
Northern Victoria,11,"{other, Ray, Raine, hockingstuart, HAR, McDona..."
Western Victoria,6,"{other, Ray, hockingstuart, Raine, HAR, YPA}"


In [43]:
# Группируем столбец по признаку количества комнат:
melb_data.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

Rooms
7     1.920700e+06
5     1.870260e+06
6     1.849366e+06
8     1.602750e+06
4     1.445282e+06
3     1.076081e+06
10    9.000000e+05
2     7.750812e+05
1     4.338245e+05
Name: Price, dtype: float64

In [ ]:
# Pегион c наименьшим стандартным отклонением по географической широте (Lattitude):
melb_data.groupby('Regionname')['Lattitude'].std().sort_values()

In [ ]:
# Cоздадим переменную с начальной датой отсчетного периода в нужном формате
date1 = pd.to_datetime('2017-05-01')
# Cоздадим переменную с конечной датой отсчетного периода в нужном формате
date2 = pd.to_datetime('2017-09-01')
# Произвoдим фильтрацию по условию попадания данных о продажах в интервал указанных дат.
mask = (date1 <= melb_data['Date']) & (melb_data['Date']<= date2)
# Сгруппируем данные по признаку SellerG и посчитаем сумму продаж для каждой компании
# Отсортируем данные по возрастанию
melb_data[mask].groupby('SellerG')['Price'].sum().sort_values(ascending=True)

## Сводные таблицы

Сводная таблица принимает на вход данные из отдельных столбцов и группирует их. В результате получается новая таблица, которая позволяет увидеть многомерное обобщение данных. Таким образом, благодаря сводным таблицам мы можем оценить зависимость между двумя и более признаками данных.

## Метод GROUPBY() как способ построения сводных таблиц

In [50]:
# Зависимость медианной цены и площади здания от числа комнат:
melb_data.groupby('Rooms')[['Price', 'BuildingArea']].median()

,Price,BuildingArea
Rooms,,
1,385000.0,107.0
2,690000.0,126.0
3,950000.0,126.0
4,1285000.0,142.0
5,1660000.0,176.0
6,1800000.0,126.0
7,1496000.0,216.5
8,1515000.0,126.0
10,900000.0,126.0


In [ ]:
# Зависимость средней цены от числа комнат и типa здания (Type):
melb_data.groupby(['Rooms', 'Type'])['Price'].mean()

Предсставим финальный результат был представлен в виде сводной таблицы (первый группировочный признак по строкам, а второй — по столбцам), а не в виде Series с иерархическими индексами, к результату чаще всего применяют метод unstack(), который позволяет переопределить вложенный индекс в виде столбцов таблицы:

In [ ]:
melb_data.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

## Метод PIVOT_TABLE() для построения сводных таблиц

На самом деле метод groupby редко используется при двух параметрах, так как для построения сводных таблиц существует специальный и более простой метод — pivot_table().

In [ ]:
# Построим ту же таблицу, но с pivot_table():
melb_data.pivot_table(
    # В качестве параметра values укажем столбец Price
    values='Price',
    # В качестве индексов сводной таблицы возьмём Rooms
    index='Rooms',
    # В качестве столбцов — Type
    columns='Type', 
    # Заменим пропуски в таблице на значение 0
    fill_value=0
).round()    

In [ ]:
# Проанализируем продажи в каждом из регионов в зависимости от того, 
# будний был день или выходной
melb_data.pivot_table(
    # В качестве параметра values укажем столбец Price
    values='Price',
    # В качестве индексов сводной таблицы возьмём Regionname
    index='Regionname',
    # В качестве столбцов — Weekend,
    # который равен 1, если день был выходным, и 0 — в противном случае
    columns='Weekend',
    # В качестве значений сводной таблицы возьмём количество продаж
    aggfunc='count'
)

In [ ]:
melb_data.pivot_table(
    # В качестве параметра values укажем столбец Landsize
    values='Landsize',
    # В качестве индексов сводной таблицы возьмём Regionname
    index='Regionname',
    # В качестве столбцов — Type,
    columns='Type',
    # В качестве значений сводной таблицы возьмём количество продаж
    aggfunc=['median', 'mean'],
    # Пропущенные значения заменим на 0
    fill_value=0
)

## Многомерные сводные таблицы

Для того чтобы исследовать зависимость от большего числа признаков, можно передать список признаков в параметр index или параметр columns.

In [ ]:
# Проанализируем
melb_data.pivot_table(
    # Ha пересечении строк и столбцов будет стоять медианная цена объекта (Price)
    values='Price',
    # В качестве индексов признаки метода продажи (Method) и типа объекта (Type)
    index=['Method', 'Type'],
    # В качестве столбцов — Regionname,
    columns='Regionname',
    # В качестве значений сводной таблицы возьмём количество продаж
    aggfunc='median',
    # Пропущенные значения заменим на 0
    fill_value=0
)

## Доступ к данным в сводной таблице

In [ ]:
# Запишем сводную таблицу, которую мы создавали ранее в переменную pivot:
pivot = melb_data.pivot_table(
    # В качестве параметра values укажем столбец Landsize
    values='Landsize',
    # В качестве индексов сводной таблицы возьмём Regionname
    index='Regionname',
    # В качестве столбцов — Type,
    columns='Type',
    # В качестве значений сводной таблицы возьмём количество продаж
    aggfunc=['median', 'mean'],
    # Пропущенные значения заменим на 0
    fill_value=0
)
# Выведем её столбцы с помощью атрибута columns:
pivot.columns

В результате мы получаем объект MultiIndex. Этот объект хранит в себе шесть комбинаций пар столбцов (два статистических параметра и три типа здания), то есть есть шесть возможных вариантов обращения к столбцам таблицы.

Мультииндексы раскрываются подобно вложенным словарям — по очереди, как матрёшка. Чтобы получить доступ к определённому столбцу, вы должны сначала обратиться к столбцу, который находится уровнем выше.

In [63]:
# Получим средние значения площадей участков для типа здания unit, 
# просто последовательно обратившись по имени столбцов:
display(pivot['mean']['unit'])

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

In [64]:
# Регионы со средней площадью здания для домов типа house меньше их медианной площади
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

median                       mean              \
Type                   house townhouse unit       house   townhouse   
Regionname                                                            
Southern Metropolitan  586.0     246.0    0  569.643881  278.858824   
Western Metropolitan   531.0     198.0   62  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

In [65]:
# Получим индексы отфильтрованной таблицы:
print(list(filtered_pivot.index))

['Southern Metropolitan', 'Western Metropolitan']


In [66]:
# Давайте создаим мультииндексные таблицы вручную:
import numpy as np

mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
display(mser)

white  up       0.034818
       down     0.447507
       right    0.722222
blue   up       0.941220
       down     0.377280
red    up       0.043097
       down     0.037214
       left     0.180002
dtype: float64

В данном примере мы создаём объект Series со вложенными индексами. Мы передаём в качестве индексов Series вложенный список, где первый список задаёт внешний уровень вложенности, а второй список — внутренний уровень вложенности. Значения Series — случайные числа от 0 до 1, сгенерированные функцией np.random.rand() (ваши значения могут отличаться).

In [ ]:
# Если посмотреть на индексы Series, можно увидеть, что они являются мультииндексами:
print(mser.index)

Аналогично создаются DataFrame со вложенными признаками (вложенными столбцами) — для этого вложенный список передаётся в параметр columns при инициализации таблицы:

In [68]:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

pen               paper          
                   1         2         1         2
white up   -0.560573 -1.464048  0.226950  0.154149
      down  1.034237  0.810785  1.062739  0.498599
red   up    0.777923 -0.933349  0.972406 -0.442110
      down -0.951736 -0.128577  1.317283  0.468895

In [69]:
melb_data.pivot_table(
    # Ha пересечении строк и столбцов будет стоять медианная площадь объекта (BuildingArea)
    values='BuildingArea',
    # В качестве индексов признаки типа объекта (Type)
    index=['Type'],
    # В качестве столбцов — Rooms,
    columns='Rooms',
    # В качестве значений сводной таблицы медианa
    aggfunc='median',
    # Пропущенные значения заменим на 0
    fill_value=0
)

Rooms,1,2,3,4,5,6,7,8,10
Type,,,,,,,,,
house,126.0,126,126,141.0,177,126,216.5,126,126
townhouse,88.0,114,126,159.5,152,0,0.0,0,0
unit,69.5,110,126,126.0,0,171,0.0,126,0


Составьте сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости (Price) от риелторского агентства (SellerG) и типа здания (Type).
Во вновь созданной таблице найдите агентство, у которого медианная цена для зданий типа unit максимальна. В качестве ответа запишите название этого агентства.

In [72]:
melb_data.pivot_table(
    # Ha пересечении строк и столбцов будет стоять медианная (Price)
    values='Price',
    # В качестве индексов признаки риелторского агентства (SellerG)
    index='SellerG',
    # В качестве столбцов типа здания (Type),
    columns='Type',
    # В качестве значений сводной таблицы медианa
    aggfunc='median',
    # Пропущенные значения заменим на 0
    fill_value=0
).sort_values(
    by='unit',
    ascending=False
)

Type,house,townhouse,unit
SellerG,,,
Nick,2025000,780000,900000
Marshall,1975000,1408500,715000
Cayzer,1505000,1450000,707500
Kay,2220000,1365000,695000
Noel,1400500,990000,693250
Buxton,1323750,1030000,670000
Fletchers,1390000,1238000,653000
Chisholm,1520000,950000,640000
Philip,1035000,701000,636000
